# Application: Financial report

## Goals of the app

This app will do the following:
* Takes as an input asset tickers
* Downloads asset price data, analyzes them (correlations, returns, ...)
* Retrieve or compute the following ratios:
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
* Analyze the correlation between the stocks
* Plot their normalized prices for comparison
* Downloads the latest news headlines about these assets and then analyzes and summarizes these news headlines
* Writes a financial report about these assets based on all this information
    * **Legal reviewer**: Check that the content is legally compliant.
    * **Text/Data alignment reviewer**: Check that the text content is aligned with the text written in the report
    * **Consistency reviewer**: Check that the conclusion are consistent throughout the text and no contradictions are present
    * **Completion reviewer**: Check that the report contains all the elements requested
* Refines the report based on previous criticisms
* Saves the final report to a mardown file with a normalized price chart


In [ ]:
# llm_config

llm_config = {
    "model": "gpt-4o", 
    "api_key": ""
    }


import autogen

## Agents definition

The app will rely on agents to accomplish its goal.

### 1. Financial & Research task

The first task will be to get the financial data about the chosen assets and compute some of their performance metrics, the second task will be to investigate reasons for this performance based on news headlines. Let's first provide a prompt for these tasks. Note that we will provide both of these prompts in a list for convenience:

In [ ]:
from datetime import datetime
date_str = datetime.now().strftime("%Y-%m-%d")

stocks = "AMD, NVDA"

financial_tasks = [
    f"""Today is the {date_str}. 
    What are the current stock prices of {stocks}, and how is the performance over the past 6 months in terms of percentage change? 
    Start by retrieving the full name of each stock and use it for all future requests.
    Prepare a figure of the normalized price of these stocks and save it to a file named normalized_prices.png. Include information about, if applicable: 
    * P/E ratio
    * Forward P/E
    * Dividends
    * Price to book
    * Debt/Eq
    * ROE
    * Analyze the correlation between the stocks
    Do not use a solution that requires an API key.
    If some of the data does not makes sense, such as a price of 0, change the query and re-try.""",

    """Investigate possible reasons of the stock performance leveraging market news headlines from Bing News or Google Search. Retrieve news headlines using python and return them. Use the full name stocks to retrieve headlines. Retrieve at least 10 headlines per stock. Do not use a solution that requires an API key.""",
]

We now need agents to perform these taks. We are going to define two agents to do so, a Financial agent, for the first task, and a Research agent, for the second task. Both agents will have to provide code that can perform each task, so these will be Assistant agents with LLM capability:

In [ ]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)

At the end of this task, we will have the price data, performance metrics and a chart that shows normalized price evolution fot the specified assets. 

### 2. Writing task

The second taks will be to write a first version of a report based on the data provided by the previous agents. Let's define the task:

In [ ]:
writing_tasks = [
        """Develop an engaging financial report using all information provided, include the normalized_prices.png figure,
        and other figures if provided.
        Mainly rely on the information provided. 
        Create a table comparing all the fundamental ratios and data.
        Provide comments and description of all the fundamental ratios and data.
        Compare the stocks, consider their correlation and risks, provide a comparative analysis of the stocks.
        Provide a summary of the recent news about each stock. 
        Ensure that you comment and summarize the news headlines for each stock, provide a comprehensive analysis of the news.
        Provide connections between the news headlines provided and the fundamental ratios.
        Provide an analysis of possible future scenarios. 
        """]

Let's define an agent to work on this task:

In [ ]:
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging finance reports.
        You transform complex concepts into compelling narratives. 
        Include all metrics provided to you as context in your analysis.
        Only answer with the financial report written in markdown directly, do not include a markdown language block indicator.
        Only return your final work without additional comments.
        """,
)

### 3. Refining the blogpost

To refine the blogpost based on SEO, legal aspects and ethical aspects, we are going to use a nested chat that will be triggered when the Writer contacts the Critic. The Critic will trigger a nested chat with agents designed to optimize each aspect. A final agent in this nested chat, the Meta Reviewer, will summarize and assemble all reviews and send them back to the Critic, who'll transfer them to the Writer agent to prepare a refined version. Let's define all these agents:

In [ ]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

consistency_reviewer = autogen.AssistantAgent(
    name="Consistency reviewer",
    llm_config=llm_config,
    system_message="You are a consistency reviewer, known for "
        "your ability to ensure that the written content is consistent throughout the report. "
        "Refer numbers and data in the report to determine which version should be chosen " 
        "in case of contradictions. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

textalignment_reviewer = autogen.AssistantAgent(
    name="Text alignment reviewer",
    llm_config=llm_config,
    system_message="You are a text data alignment reviewer, known for "
        "your ability to ensure that the meaning of the written content is aligned "
        "with the numbers written in the text. " 
        "You must ensure that the text clearely describes the numbers provided in the text "
        "without contradictions. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

completion_reviewer = autogen.AssistantAgent(
    name="Completion Reviewer",
    llm_config=llm_config,
    system_message="You are a content completion reviewer, known for "
        "your ability to check that financial reports contain all the required elements. "
        "You always verify that the report contains: a news report about each asset, " 
        "a description of the different ratios and prices, "
        "a description of possible future scenarios, a table comparing fundamental ratios and "
        " at least a single figure. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggregate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

### 4. Exporting the blogpost in markdown

We will define one agent that will write the python code necessary to export the final blogpost in a markdown file:

In [ ]:
exporting_task = ["""Save the blogpost and only the blogpost to a .md file using a python script."""]

In [ ]:
export_assistant = autogen.AssistantAgent(
    name="Exporter",
    llm_config=llm_config,
)

### Code execution

Finally, we will need an agent to execute code. Since we're preparing an app that will have to run without human intervention, we will create a user proxy agent that will never request human input. This is a bit risky as the agents might get stuck in an execution loop of code that does not work; they might also execute malicious code (the risk is very low for this app). Let's define such an agent:

In [ ]:
user_proxy_auto = autogen.UserProxyAgent(
    name="User_Proxy_Auto",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },  
)

## Agentic flow

We are now going to define the flow of chats through which our agents will interact. We will have to define two chats:
1. The nested chat flow for blog post writing refining
2. The main chat flow

### Nested chat flow

Let's first start by defining the nested chat flow. This chat will be triggered when the Writer will contact the Critic:

In [ ]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": textalignment_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    {"recipient": consistency_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
    {"recipient": completion_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into a JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]

critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

### Main chat flow

Let's now define the main chat flow, it will be composed of 4 chats:
1. Financial agent with user proxy to accomplish the first financial task
2. Research agent with user proxy to accomplish the second financial task
3. Critic with Writer to write and refine the blogpost through the Nested chat
4. Export agent with user proxy to export the markdown blogpost to a file

In [ ]:
chat_results = autogen.initiate_chats(
    [
        {
            "sender": user_proxy_auto,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "silent": False,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : "Return the stock prices of the stocks, their performance and all other metrics"
                "into a JSON object only. Provide the name of all figure files created. Provide the full name of each stock.",
                            },
            "clear_history": False,
            "carryover": "Wait for confirmation of code execution before terminating the conversation. Verify that the data is not completely composed of NaN values. Reply TERMINATE in the end when everything is done."
        },
        {
            "sender": user_proxy_auto,
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "silent": False,
            "summary_method": "reflection_with_llm",
            "summary_args": {
                "summary_prompt" : "Provide the news headlines as a paragraph for each stock, be precise but do not consider news events that are vague, return the result as a JSON object only.",
                            },
            "clear_history": False,
            "carryover": "Wait for confirmation of code execution before terminating the conversation. Reply TERMINATE in the end when everything is done."
        },
        {
            "sender": critic,
            "recipient": writer,
            "message": writing_tasks[0],
            "carryover": "I want to include a figure and a table of the provided data in the financial report.",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "sender": user_proxy_auto,
            "recipient": export_assistant,
            "message": exporting_task[0],
            "carryover": "Wait for confirmation of code execution before terminating the conversation. Reply TERMINATE in the end when everything is done.",
        }
    ]
)

We can execute the code from the .py script created with coding part.  

`python RA-Application-financial_report.py`